In [1]:
from pandas import read_csv
import numpy as np
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [16]:
def answer(filename, *answer):
    file = open(filename, 'w')
    ans = ''
    for i in answer:
        ans += str(i) + ' '
    ans = ans[:-1]
    file.write(ans)
    file.close()

In [32]:
train   = read_csv('perceptron-train.csv', header=None)
train_y = train.iloc[:,0]
train_x = train.iloc[:,[1,2]]


test = read_csv('perceptron-test.csv', header=None)
test_y = train.iloc[:,0]
test_x = train.iloc[:,[1,2]]

In [36]:
model = Perceptron(random_state=241)
model.fit(train_x, train_y)
predictions = model.predict(test_x)
accuracy_score(test_y, predictions)

0.7033333333333334

In [35]:
#нормализация признаков
scaler         = StandardScaler()
train_x_scaler = scaler.fit_transform(train_x)
model.fit(train_x_scaler, train_y)
test_x_scaler = scaler.fit_transform(test_x)
predict_scaler = model.predict(test_x_scaler)
accuracy_score(test_y, predict_scaler)


0.77

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from scipy.sparse import hstack
from sklearn.linear_model import Ridge

In [5]:
train = pd.read_csv('salary-train.csv')
test  = pd.read_csv('salary-test-mini.csv')

In [9]:
#Убираем грязь
def transformData(data):
    data.FullDescription = data.FullDescription.replace('[^a-zA-Z0-9]', ' ', regex=True).str.lower()
    return data

In [13]:
train = transformData(train)
test  = transformData(test)

In [32]:
# преобразуем тексты в числа ахаха
vectorizer = TfidfVectorizer(min_df=5)
tf_idf  = vectorizer.fit_transform(train.FullDescription)


In [33]:
train['LocationNormalized'].fillna('nan', inplace=True)
train['ContractTime'].fillna('nan', inplace=True)
train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,London,permanent,33000
1,an ideal opportunity for an individual that ha...,London,permanent,50000
2,online content and brand manager luxury reta...,South East London,permanent,40000
3,a great local marketleader is seeking a perman...,Dereham,permanent,22500
4,registered nurse rgn nursing home for young...,Sutton Coldfield,nan,20355


In [34]:
one_hot = DictVectorizer()

# one hot transformation for text fields
oh = one_hot.fit_transform(train[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [35]:
X = hstack([tf_idf, oh])
y = train['SalaryNormalized']

In [36]:
clf = Ridge(alpha=1, random_state=241)
clf.fit(X, y)

Ridge(alpha=1, random_state=241)

In [37]:
tf_idf_test = vectorizer.transform(test.FullDescription)
oh_test = one_hot.transform(test[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test = hstack([tf_idf_test, oh_test])

res = clf.predict(X_test)
print(res)

f = open('submission.txt', 'w')
f.write(str(res)[1:-1])
f.close()


[56565.3254579  37140.63063337]


In [10]:
from sklearn.decomposition import PCA
data = pd.read_csv('close_prices.csv')

dataUseFull = data.iloc[:,1:]
dataUseFull

,AXP,BA,CAT,CSCO,CVX,DD,DIS,GE,GS,HD,...,PFE,PG,T,TRV,UNH,UTX,V,VZ,WMT,XOM
0,76.440002,117.510002,85.029999,24.270000,125.519997,59.409999,64.750000,24.280001,165.250000,75.910004,...,28.799999,79.279999,34.220001,86.379997,71.820000,109.419998,196.240005,47.980000,76.419998,87.750000
1,76.070000,119.000000,85.110001,24.139999,124.489998,59.319997,64.320000,24.320000,162.970001,76.040001,...,28.709999,78.620003,34.090000,85.870003,72.320000,110.000000,193.339996,47.270000,75.750000,87.360001
2,75.989998,118.510002,84.500000,24.430000,124.070000,59.319997,64.449997,24.230000,162.309998,75.519997,...,28.490000,77.720001,34.049999,85.980003,71.980003,109.260002,191.559998,46.950001,74.650002,87.139999
3,76.320000,119.379997,84.199997,23.770000,123.489998,59.509996,65.239998,24.250000,162.289993,76.070000,...,28.520000,78.050003,34.230000,85.830002,72.160004,109.660004,193.559998,47.669998,74.620003,87.070000
4,75.889999,118.739998,83.800003,23.330000,122.639999,59.009995,65.190002,24.049999,159.850006,75.959999,...,28.879999,77.209999,33.980000,85.410004,71.989998,109.360001,193.050003,47.000000,74.360001,86.900002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,81.559998,152.039993,80.190002,28.240000,102.419998,80.400001,107.169998,25.400000,189.949997,116.099998,...,34.000000,82.089996,33.130001,107.830002,114.879997,121.239998,269.579987,48.730000,81.900002,84.220001
370,80.599998,151.570007,79.230003,27.940001,101.620003,80.499997,106.440002,25.040001,189.339996,114.820000,...,34.000000,81.830002,32.759998,106.720001,115.250000,118.739998,265.029999,48.840000,81.900002,83.870003
371,81.500000,153.669998,79.970001,28.299999,103.129997,77.069994,107.370003,25.450001,191.899994,116.500000,...,34.439999,83.559998,33.060001,108.930000,118.519997,120.070000,269.019989,49.270000,83.290001,84.760002
372,81.059998,154.509995,78.449997,28.150000,103.169998,74.680001,106.959999,25.309999,190.570007,115.580002,...,34.169998,82.849998,32.900002,108.750000,117.599998,119.860001,264.500000,48.950001,82.620003,84.080002


In [14]:
pca = PCA(n_components=10)
dataTrans = pca.fit_transform(dataUseFull)
dispersionComp = pca.explained_variance_ratio_

In [17]:
component_dispersion = pca.explained_variance_ratio_
i = 1
while np.sum(component_dispersion[:i]) < 0.9:
    i += 1
print (i)
answer('4-2-1.txt', i)

4


In [19]:
first_component = dataTrans[:, 0]
first_component

array([-50.90240358, -52.84690919, -54.61443917, -52.60056628,
       -52.3701233 , -54.65341197, -52.81257496, -53.6511457 ,
       -56.69272698, -54.40265506, -58.33164549, -61.71626498,
       -61.31940025, -53.79252445, -50.89263448, -49.63740755,
       -52.3876147 , -47.47854294, -43.56033499, -42.55928046,
       -42.45027087, -42.27124797, -43.25272816, -40.05934481,
       -38.49726315, -39.06492263, -38.39304378, -38.40055833,
       -42.38616981, -39.83057417, -41.29548604, -40.43554958,
       -38.49607932, -42.96158052, -38.71252143, -38.83379331,
       -39.90606174, -37.58824522, -34.92356725, -32.96232694,
       -34.52461722, -36.40345948, -37.22194659, -32.95883861,
       -30.59159319, -29.63686919, -27.34513869, -26.90264007,
       -27.41985754, -28.24277351, -31.79501246, -31.39698179,
       -33.14502283, -30.41497345, -30.73495319, -32.3505575 ,
       -30.84313809, -32.84916115, -30.36248061, -29.89452878,
       -25.10690112, -19.92249325, -19.83315396, -19.22

In [25]:
dj = pd.read_csv('djia_index.csv')
corr = np.corrcoef(first_component, dj.iloc[:,1])[0,1]
answer('4-2-2.txt', corr)

In [29]:
company = dataUseFull.keys()[pca.components_[0].argmax()]

answer('4-2-3.txt', company)